In [10]:
from __future__ import print_function

import numpy as np
import pandas as pd
import time

np.random.seed(2)  # reproducible

N_STATES = 10 
ACTIONS = ['left', 'right']  
EPSILON = 0.9 
ALPHA = 0.1 
GAMMA = 0.9 
MAX_EPISODES = 13
FRESH_TIME = 0.3 

In [11]:
def build_q_table(n_states, actions):
    table = pd.DataFrame(
        np.zeros((n_states, len(actions))), 
        columns=actions,
    )
    return table

In [12]:
def choose_action(state, q_table):
    state_actions = q_table.iloc[state, :]  
    if (np.random.uniform() > EPSILON) or (state_actions.all() == 0):
        action_name = np.random.choice(ACTIONS)
    else:
        action_name = state_actions.argmax()
    return action_name

In [13]:

def get_env_feedback(S, A):
    # This is how agent will interact with the environment
    if A == 'right':    # move right
        if S == N_STATES - 2:   # terminate
            S_ = 'terminal'
            R = 1
        else:
            S_ = S + 1
            R = 0
    else:   # move left
        R = 0
        if S == 0:
            S_ = S  # reach the wall
        else:
            S_ = S - 1
    return S_, R

In [14]:

def update_env(S, episode, step_counter):
    # This is how environment be updated
    env_list = ['-']*(N_STATES-1) + ['T'] 
    if S == 'terminal':
        interaction = 'Episode %s: total_steps = %s' % (episode+1, step_counter)
        print('\r{}'.format(interaction), end='')
        time.sleep(2)
        print('\r                                ', end='')
    else:
        env_list[S] = 'o'
        interaction = ''.join(env_list)
        print('\r{}'.format(interaction), end='')
        time.sleep(FRESH_TIME)

In [15]:

def rl():
    q_table = build_q_table(N_STATES, ACTIONS) 
    for episode in range(MAX_EPISODES):  
        step_counter = 0
        S = 0 
        is_terminated = False  
        update_env(S, episode, step_counter)  
        while not is_terminated:

            A = choose_action(S, q_table) 
            S_, R = get_env_feedback(S, A) 
            q_predict = q_table.ix[S, A] 
            if S_ != 'terminal':
                q_target = R + GAMMA * q_table.iloc[S_, :].max() 
            else:
                q_target = R  
                is_terminated = True  

            q_table.ix[S, A] += ALPHA * (q_target - q_predict)
            S = S_ 

            update_env(S, episode, step_counter+1) 

            step_counter += 1
    return q_table

In [16]:
if __name__ == "__main__":
    q_table = rl()
    print('\r\nQ-table:\n')
    print(q_table)

o--------T

/usr1/home/bohanl1/.local/lib/python2.7/site-packages/ipykernel_launcher.py:13: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  del sys.path[0]


                                
Q-table:

           left         right
0  2.061077e-10  8.036376e-07
1  8.813455e-09  1.355705e-05
2  3.573249e-07  1.924061e-04
3  1.715274e-05  1.007605e-03
4  4.666368e-05  6.253090e-03
5  2.184813e-06  3.333784e-02
6  1.755662e-03  1.224253e-01
7  7.290000e-05  3.631579e-01
8  1.795456e-02  7.458134e-01
9  0.000000e+00  0.000000e+00
